In [9]:
from pyfaidx import Fasta
import numpy as np
import pandas as pd
import os
from random import choice, seed
from itertools import izip

meta_info = pd.read_table('./metadata_update/Simons_meta_accessionnb_update.txt')
#print meta_info.columns.values

cteam_id_list = meta_info.loc[meta_info['Gender'] == 'M']['CTeam_ID'].values

path = '/home/kmt/simons/faststorage/cteam_lite_public3/FullyPublic/{}.fa'
sample_id_list = [x for x in cteam_id_list if os.path.exists(path.format(x)) and x.startswith('S')]

# index for all genomes:
#samples = dict((x, Fasta(path.format(x))) for x in sample_id_list)

chrX_samples = dict()
for smpl in sample_id_list:
    with open(path.format(x)) as f:
        s = f.read()
        for entry in s.split('>'):
            if not entry: 
                continue            
            header, seq = entry.split('\n', 1)
            chrom = header.split()[0]
            if chrom == 'X':
                chrX_samples[smpl] = seq.replace('\n', '')
                break


155270560


In [16]:
seed(7)

iupac = {'A': 'A',
'C': 'C',
'G': 'G',
'T': 'T',
'R': 'AG',
'Y': 'CT',
'S': 'GC',
'W': 'AT',
'K': 'GT',
'M': 'AC',
'B': 'CGT',
'D': 'AGT',
'H': 'ACT',
'V': 'ACG',
'N': 'ATGC' }

def dist(x, y):
    uncalled, same, diff = 0, 0, 0
    assert len(x) == len(y), (x, y)
    for a, b in izip(x, y):
        if a in '-N' or b in '-N':
            uncalled += 1
        elif a == b: # test explicitly to speed up
            same += 1
        elif choice(iupac[a]) != choice(iupac[b]):
            diff += 1
        else:
            same += 1
    assert uncalled + same + diff == len(x)
    if len(x) == uncalled:
        return float('nan')
    else:
        return diff / float(len(x) - uncalled)


In [ ]:
chrX_length = 155270560
chrom = 'X'
win_size = 100000

with open('dist_matrix_in_windows.tsv', 'w') as outf:
    for idx in range(0, chrX_length, win_size):
#        seq_list = [str(samples[x][chrom][idx:idx+win_size]) for x in sample_id_list]
        seq_list = [chrX_samples[x][idx:idx+win_size] for x in sample_id_list]
        matrix = list()
        cache = dict()
        for i in range(len(seq_list)):
            for j in range(len(seq_list)):
                key = tuple(sorted([i, j]))
                if key not in cache:
                    cache[key] = dist(seq_list[i], seq_list[j])
                matrix.append(cache[key])
#                 matrix.append(dist(str(samples[s1][chrom][i:i+win_size]), 
#                                    str(samples[s2][chrom][i:i+win_size])))
        print >>outf, '\t'.join(map(str, [chrom, idx, idx+win_size] + matrix))

